In [1]:
from laserembeddings import Laser

laser = Laser()

# if all sentences are in the same language:

embeddings = laser.embed_sentences(
    ['let your neural network be polyglot',
     'biarkan jaringan saraf Anda menjadi poliglot'],
    lang=['en', 'id', 'in'])  # lang is only used for tokenization

# embeddings is a N*1024 (N = number of sentences) NumPy array
print(embeddings)

[[0.00156426 0.01067378 0.00388247 ... 0.02179701 0.0062856  0.01434554]
 [0.00023719 0.00806764 0.01312093 ... 0.00290587 0.01923944 0.02042993]]


In [2]:
from scipy import spatial
dist_1 = spatial.distance.cosine(embeddings[0], embeddings[1])
dist_1

0.11816513538360596

In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.set_option('display.max_colwidth', None)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
from gensim.models import Word2Vec
import nltk
nltk.download('wordnet')
stemmer = SnowballStemmer('english')

from numpy import dot
from numpy.linalg import norm

/Users/sibish/Documents/OMSCS/CS 7643/env/laser/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
[nltk_data] Downloading package wordnet to /Users/sibish/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
train_df = pd.read_csv('train.csv')
tmp = train_df.groupby('label_group').posting_id.agg('unique').to_dict()
train_df['target'] = train_df.label_group.map(tmp)
train_df
list = train_df[train_df['posting_id'] == 'train_129225211']['target'][0]
list = ' '.join(list)
list
def target_string(list):
    return ' '.join(list)
    
train_df['target_string'] = train_df.apply(lambda x: target_string(x['target']),axis=1)
train_df.head(2)

,posting_id,image,image_phash,title,label_group,target,target_string
0,train_129225211,0000a68812bc7e98c42888dfb1c07da0.jpg,94974f937d4c2433,Paper Bag Victoria Secret,249114794,"[train_129225211, train_2278313361]",train_129225211 train_2278313361
1,train_3386243561,00039780dfc94d01db8676fe789ecd05.jpg,af3f9460c2838f0f,"Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DOUBLE FOAM TAPE",2937985045,"[train_3386243561, train_3423213080]",train_3386243561 train_3423213080


In [5]:
train_df_val = train_df.head(10)

In [6]:
embeddings_list = []

In [7]:
for match_index, match_row in train_df_val.iterrows():
    print(match_row['title']),
    embeddings = laser.embed_sentences(
     [match_row['title']],
     lang=['en', 'id', 'in']),
    print(type(embeddings[0])),
    embeddings_list.append(embeddings[0])

Paper Bag Victoria Secret
<class 'numpy.ndarray'>
Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DOUBLE FOAM TAPE
<class 'numpy.ndarray'>
Maling TTS Canned Pork Luncheon Meat 397 gr
<class 'numpy.ndarray'>
Daster Batik Lengan pendek - Motif Acak / Campur - Leher Kancing (DPT001-00) Batik karakter Alhadi
<class 'numpy.ndarray'>
Nescafe \xc3\x89clair Latte 220ml
<class 'numpy.ndarray'>
CELANA WANITA  (BB 45-84 KG)Harem wanita (bisa cod)
<class 'numpy.ndarray'>
Jubah anak size 1-12 thn
<class 'numpy.ndarray'>
KULOT PLISKET SALUR /CANDY PLISKET /WISH KULOT PREMIUM /KULOT PELANGI PREMIUM/HIEKA KULOT
<class 'numpy.ndarray'>
[LOGU] Tempelan kulkas magnet angka, tempelan angka magnet
<class 'numpy.ndarray'>
BIG SALE SEPATU PANTOFEL KULIT KEREN KERJA KANTOR LAKI PRIA COWOK DINAS RESMI FORMAL PESTA KICKERS
<class 'numpy.ndarray'>


In [11]:
def find_similarity(embedding1, embedding2):
    dist_1 = spatial.distance.cosine(embedding1, embedding2)
    return dist_1

In [13]:
for match_index, match_row in train_df_val.iterrows():
    print(match_row['title'])
    match_title=match_row['title']
    match_embedding = embeddings_list[match_index]
    matched = ""
    for index, row in train_df_val.iterrows():
        current_title=row['title']
        current_embedding = embeddings_list[index]
        sim = find_similarity(match_embedding, current_embedding )
        if sim < 0.1:
            print('matched::',match_title, current_title, sim)
            if len(matched)< 1:
              matched = row['posting_id']
            else:
              matched = matched + " " + row['posting_id']
    print("matched ", matched)  
    train_df_val.loc[match_index,'predicted'] = matched

Paper Bag Victoria Secret
matched:: Paper Bag Victoria Secret Paper Bag Victoria Secret 0.0
matched  train_129225211
Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DOUBLE FOAM TAPE
matched:: Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DOUBLE FOAM TAPE Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DOUBLE FOAM TAPE 0.0
matched  train_3386243561
Maling TTS Canned Pork Luncheon Meat 397 gr
matched:: Maling TTS Canned Pork Luncheon Meat 397 gr Maling TTS Canned Pork Luncheon Meat 397 gr 0.0
matched  train_2288590299
Daster Batik Lengan pendek - Motif Acak / Campur - Leher Kancing (DPT001-00) Batik karakter Alhadi
matched:: Daster Batik Lengan pendek - Motif Acak / Campur - Leher Kancing (DPT001-00) Batik karakter Alhadi Daster Batik Lengan pendek - Motif Acak / Campur - Leher Kancing (DPT001-00) Batik karakter Alhadi 0.0
matched  train_2406599165
Nescafe \xc3\x89clair Latte 220ml
matched:: Nescafe \xc3\x89clair Latte 220ml Nescafe \xc3\x89clair Latte 220ml 0.0
matched  train_3369186413
CELANA WA

/Users/sibish/Documents/OMSCS/CS 7643/env/laser/lib/python3.7/site-packages/pandas/core/indexing.py:1599: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/Users/sibish/Documents/OMSCS/CS 7643/env/laser/lib/python3.7/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [14]:
train_df_val

,posting_id,image,image_phash,title,label_group,target,target_string,predicted
0,train_129225211,0000a68812bc7e98c42888dfb1c07da0.jpg,94974f937d4c2433,Paper Bag Victoria Secret,249114794,"[train_129225211, train_2278313361]",train_129225211 train_2278313361,train_129225211
1,train_3386243561,00039780dfc94d01db8676fe789ecd05.jpg,af3f9460c2838f0f,"Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DOUBLE FOAM TAPE",2937985045,"[train_3386243561, train_3423213080]",train_3386243561 train_3423213080,train_3386243561
2,train_2288590299,000a190fdd715a2a36faed16e2c65df7.jpg,b94cb00ed3e50f78,Maling TTS Canned Pork Luncheon Meat 397 gr,2395904891,"[train_2288590299, train_3803689425]",train_2288590299 train_3803689425,train_2288590299
3,train_2406599165,00117e4fc239b1b641ff08340b429633.jpg,8514fc58eafea283,Daster Batik Lengan pendek - Motif Acak / Campur - Leher Kancing (DPT001-00) Batik karakter Alhadi,4093212188,"[train_2406599165, train_3342059966]",train_2406599165 train_3342059966,train_2406599165
4,train_3369186413,00136d1cf4edede0203f32f05f660588.jpg,a6f319f924ad708c,Nescafe \xc3\x89clair Latte 220ml,3648931069,"[train_3369186413, train_921438619]",train_3369186413 train_921438619,train_3369186413
5,train_2464356923,0013e7355ffc5ff8fb1ccad3e42d92fe.jpg,bbd097a7870f4a50,CELANA WANITA (BB 45-84 KG)Harem wanita (bisa cod),2660605217,"[train_2464356923, train_2753295474, train_305884580]",train_2464356923 train_2753295474 train_305884580,train_2464356923
6,train_1802986387,00144a49c56599d45354a1c28104c039.jpg,f815c9bb833ab4c8,Jubah anak size 1-12 thn,1835033137,"[train_1802986387, train_1396161074, train_713073906, train_1275191373, train_2490201622, train_2411544001, train_1859060005]",train_1802986387 train_1396161074 train_713073906 train_1275191373 train_2490201622 train_2411544001 train_1859060005,train_1802986387
7,train_1806152124,0014f61389cbaa687a58e38a97b6383d.jpg,eea7e1c0c04da33d,KULOT PLISKET SALUR /CANDY PLISKET /WISH KULOT PREMIUM /KULOT PELANGI PREMIUM/HIEKA KULOT,1565741687,"[train_1806152124, train_3227306976]",train_1806152124 train_3227306976,train_1806152124
8,train_86570404,0019a3c6755a194cb2e2c12bfc63972e.jpg,ea9af4f483249972,"[LOGU] Tempelan kulkas magnet angka, tempelan angka magnet",2359912463,"[train_86570404, train_2837452969, train_77364776]",train_86570404 train_2837452969 train_77364776,train_86570404
9,train_831680791,001be52b2beec40ddc1d2d7fc7a68f08.jpg,e1ce953d1a70618f,BIG SALE SEPATU PANTOFEL KULIT KEREN KERJA KANTOR LAKI PRIA COWOK DINAS RESMI FORMAL PESTA KICKERS,2630990665,"[train_831680791, train_3031035861]",train_831680791 train_3031035861,train_831680791
